In [1]:
import pandas as pd
import re
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import wordnet
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import textattack
from textattack.models.wrappers.model_wrapper import ModelWrapper
from textattack.transformations.word_swaps import WordSwap
from textattack import Attack
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.attack_recipes import AttackRecipe
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attacker import AttackArgs


In [2]:
def load_data(dataset_split="train"):
    df = pd.read_csv(f'LabelEncoded_{dataset_split}_data.csv')
    # Remove non-alphanumeric characters
    df["text"] = df["text"].apply(lambda x: re.sub("[^a-zA-Z]", " ", str(x)))
    df['text'] = df['text'].apply(lambda x: re.sub('  +', ' ', str(x)))
    df['text'] = df['text'].apply(lambda x: re.sub('\n', ' ', str(x)))
    df['text'] = df['text'].apply(lambda x: re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', str(x)))
    # Tokenize the training and testing data
    df_tokenized = tokenize_review(df)
    return df_tokenized

def tokenize_review(df):
    # Tokenize Reviews in the DataFrame
    tokened_reviews = [word_tokenize(rev) for rev in df["text"]]
    
    # Initialize the Indonesian stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Stem tokens for each review in Indonesian
    stemmed_tokens = []
    for tokens in tokened_reviews:
        # Join the tokens into a single string for stemming, since Sastrawi expects a sentence
        sentence = " ".join(tokens)
        # Perform stemming on the whole sentence
        stemmed_sentence = stemmer.stem(sentence)
        stemmed_tokens.append(stemmed_sentence)

    # Insert the stemmed reviews into the DataFrame
    df.insert(1, column="Stemmed", value=stemmed_tokens)
    
    return df

In [3]:
df_train = load_data("train")
df_test = load_data('test')

In [4]:
df_train

,text,Stemmed,label,source
0,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini milik oleh usaha pabrik tahu yang s...,2,indonlue/smsa
1,mohon ulama lurus dan k mmbri hujjah partai ap...,mohon ulama lurus dan k mmbri hujjah partai ap...,1,indonlue/smsa
2,lokasi strategis di jalan sumatera bandung tem...,lokasi strategis di jalan sumatera bandung tem...,2,indonlue/smsa
3,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,2,indonlue/smsa
4,duh jadi mahasiswa jangan sombong dong kasih k...,duh jadi mahasiswa jangan sombong dong kasih k...,0,indonlue/smsa
...,...,...,...,...
10995,tidak kecewa,tidak kecewa,2,indonlue/smsa
10996,enak rasa masakan nya apalagi kepiting yang me...,enak rasa masakan nya apalagi kepiting yang se...,2,indonlue/smsa
10997,hormati partai partai yang telah berkoalisi,hormat partai partai yang telah koalisi,1,indonlue/smsa
10998,pagi pagi di tol pasteur sudah macet parah bik...,pagi pagi di tol pasteur sudah macet parah bik...,0,indonlue/smsa


In [5]:
def add_augmenting_features(df):
    tokened_reviews = [word_tokenize(rev) for rev in df["text"]]
    # Create feature that measures length of reviews
    len_tokens = []
    for i in range(len(tokened_reviews)):
        len_tokens.append(len(tokened_reviews[i]))
    len_tokens = preprocessing.scale(len_tokens)
    df.insert(0, column="Lengths", value=len_tokens)

    # Create average word length (training)
    Average_Words = [len(x) / (len(x.split())) for x in df["text"].tolist()]
    Average_Words = preprocessing.scale(Average_Words)
    df["averageWords"] = Average_Words
    return df

In [6]:
df_train = add_augmenting_features(df_train)
df_test = add_augmenting_features(df_test)

In [7]:
df_train

,Lengths,text,Stemmed,label,source,averageWords
0,1.456633,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini milik oleh usaha pabrik tahu yang s...,2,indonlue/smsa,1.007417
1,-0.594078,mohon ulama lurus dan k mmbri hujjah partai ap...,mohon ulama lurus dan k mmbri hujjah partai ap...,1,indonlue/smsa,-1.261937
2,-0.054417,lokasi strategis di jalan sumatera bandung tem...,lokasi strategis di jalan sumatera bandung tem...,2,indonlue/smsa,-0.141996
3,-0.755976,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,2,indonlue/smsa,-0.436717
4,0.161447,duh jadi mahasiswa jangan sombong dong kasih k...,duh jadi mahasiswa jangan sombong dong kasih k...,0,indonlue/smsa,0.027469
...,...,...,...,...,...,...
10995,-1.457535,tidak kecewa,tidak kecewa,2,indonlue/smsa,-0.436717
10996,-0.270282,enak rasa masakan nya apalagi kepiting yang me...,enak rasa masakan nya apalagi kepiting yang se...,2,indonlue/smsa,0.182197
10997,-1.241671,hormati partai partai yang telah berkoalisi,hormat partai partai yang telah koalisi,1,indonlue/smsa,1.488795
10998,-0.917875,pagi pagi di tol pasteur sudah macet parah bik...,pagi pagi di tol pasteur sudah macet parah bik...,0,indonlue/smsa,-1.399474


In [8]:
df_test

,Lengths,text,Stemmed,label,source,averageWords
0,1.588254,kemarin gue datang ke tempat makan baru yang a...,kemarin gue datang ke tempat makan baru yang a...,0,indonlue/smsa,-1.356389
1,2.160332,kayak nya sih gue tidak akan mau balik lagi ke...,kayak nya sih gue tidak akan mau balik lagi ke...,0,indonlue/smsa,-1.072196
2,0.852724,kalau dipikir pikir sebenarnya tidak ada yang ...,kalau pikir pikir benar tidak ada yang bisa ba...,0,indonlue/smsa,0.632503
3,1.833430,ini pertama kalinya gua ke bank buat ngurusin ...,ini pertama kali gua ke bank buat ngurusin bua...,0,indonlue/smsa,-0.271788
4,1.669979,waktu sampai dengan gue pernah disuruh ibu lat...,waktu sampai dengan gue pernah suruh ibu latih...,0,indonlue/smsa,-0.306821
...,...,...,...,...,...,...
495,-0.945237,kata nya tidur yang baik itu minimal enam jam ...,kata nya tidur yang baik itu minimal enam jam ...,1,indonlue/smsa,-1.694131
496,-1.272139,indonesia itu ada di benua asia,indonesia itu ada di benua asia,1,indonlue/smsa,-1.634185
497,-0.945237,salah satu kegemaran anak remaja indonesia sek...,salah satu gemar anak remaja indonesia sekaran...,1,indonlue/smsa,2.082433
498,-1.026963,melihat warna hijau bisa bikin mata jadi lebih...,lihat warna hijau bisa bikin mata jadi lebih s...,2,indonlue/smsa,-0.435276


In [9]:
factory = StopWordRemoverFactory()
def transform_BOW(training, testing, column_name):
    vect = CountVectorizer(
        max_features=100, ngram_range=(1, 3), stop_words=list(factory.get_stop_words())
    )
    vectFit = vect.fit(training[column_name])
    BOW_training = vectFit.transform(training[column_name])
    BOW_training_df = pd.DataFrame(
        BOW_training.toarray(), columns=vect.get_feature_names_out()
    )
    BOW_testing = vectFit.transform(testing[column_name])
    BOW_testing_Df = pd.DataFrame(
        BOW_testing.toarray(), columns=vect.get_feature_names_out()
    )
    return vectFit, BOW_training_df, BOW_testing_Df


def transform_tfidf(training, testing, column_name):
    Tfidf = TfidfVectorizer(
        ngram_range=(1, 3), max_features=100, stop_words=list(factory.get_stop_words())
    )
    Tfidf_fit = Tfidf.fit(training[column_name])
    Tfidf_training = Tfidf_fit.transform(training[column_name])
    Tfidf_training_df = pd.DataFrame(
        Tfidf_training.toarray(), columns=Tfidf.get_feature_names_out()
    )
    Tfidf_testing = Tfidf_fit.transform(testing[column_name])
    Tfidf_testing_df = pd.DataFrame(
        Tfidf_testing.toarray(), columns=Tfidf.get_feature_names_out()
    )
    return Tfidf_fit, Tfidf_training_df, Tfidf_testing_df

In [10]:
# Create BOW features for training set
unstemmed_BOW_vect_fit, df_train_bow_unstem, df_test_bow_unstem = transform_BOW(
    df_train, df_test, "Stemmed"
)
print("...successfully created the unstemmed BOW data")

# Create TfIdf features for training set
unstemmed_tfidf_vect_fit, df_train_tfidf_unstem, df_test_tfidf_unstem = transform_tfidf(
    df_train, df_test, "Stemmed"
)
print("...successfully created the unstemmed TFIDF data")

...successfully created the unstemmed BOW data
...successfully created the unstemmed TFIDF data


In [13]:
def build_model(X_train, y_train, X_test, y_test, name_of_test):
    log_reg = LogisticRegression(C=30, max_iter=500).fit(X_train, y_train)
    y_pred = log_reg.predict(X_test)
    print(
        "Training accuracy of " + name_of_test + ": ", log_reg.score(X_train, y_train)
    )
    print("Testing accuracy of " + name_of_test + ": ", log_reg.score(X_test, y_test))
    print(classification_report(y_test, y_pred))  # Evaluating prediction ability
    return log_reg

In [14]:
bow_unstemmed = build_model(
    df_train_bow_unstem,
    df_train["label"],
    df_test_bow_unstem,
    df_test["label"],
    "BOW Unstemmed",
)

tfidf_unstemmed = build_model(
    df_train_tfidf_unstem,
    df_train["label"],
    df_test_tfidf_unstem,
    df_test["label"],
    "TFIDF Unstemmed",
)

Training accuracy of BOW Unstemmed:  0.7682727272727272
Testing accuracy of BOW Unstemmed:  0.564
              precision    recall  f1-score   support

           0       0.50      0.85      0.63       204
           1       0.59      0.26      0.36        88
           2       0.77      0.41      0.53       208

    accuracy                           0.56       500
   macro avg       0.62      0.51      0.51       500
weighted avg       0.63      0.56      0.54       500

Training accuracy of TFIDF Unstemmed:  0.7695454545454545
Testing accuracy of TFIDF Unstemmed:  0.544
              precision    recall  f1-score   support

           0       0.49      0.78      0.60       204
           1       0.53      0.20      0.30        88
           2       0.67      0.46      0.54       208

    accuracy                           0.54       500
   macro avg       0.56      0.48      0.48       500
weighted avg       0.57      0.54      0.52       500



In [24]:
clf = MultinomialNB()
clf.fit(df_train_tfidf_unstem, df_train['label'])

MultinomialNB()

In [26]:
y_test = clf.predict(df_test_tfidf_unstem)
print(classification_report(y_test, df_test['label']))

              precision    recall  f1-score   support

           0       0.61      0.50      0.55       250
           1       0.03      0.60      0.06         5
           2       0.57      0.49      0.53       245

    accuracy                           0.49       500
   macro avg       0.40      0.53      0.38       500
weighted avg       0.58      0.49      0.53       500



In [27]:
class SklearnModelWrapperNew(ModelWrapper):
    """Loads a scikit-learn model and tokenizer (tokenizer implements
    `transform` and model implements `predict_proba`).

    May need to be extended and modified for different types of
    tokenizers.
    """

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, text_input_list, batch_size=None):
        encoded_text_matrix = self.tokenizer.transform(text_input_list).toarray()
        tokenized_text_df = pd.DataFrame(
            encoded_text_matrix, columns=self.tokenizer.get_feature_names_out()
        )
        return self.model.predict_proba(tokenized_text_df)

def get_grad(self, text_input):
        raise NotImplementedError()

In [28]:
model_wrapper = SklearnModelWrapperNew(clf, unstemmed_tfidf_vect_fit)

In [29]:
class WordSwapWordNetBahasa(WordSwap):
    """Transforms an input by replacing its words with synonyms provided by
    WordNet.

    >>> from textattack.transformations import WordSwapWordNet
    >>> from textattack.augmentation import Augmenter

    >>> transformation = WordSwapWordNet()
    >>> augmenter = Augmenter(transformation=transformation)
    >>> s = 'I am fabulous.'
    >>> augmenter.augment(s)
    """

    def __init__(self, language="ind"):
        nltk.download("omw-1.4")
        # if language not in wordnet.langs():
        #     raise ValueError(f"Language {language} not one of {wordnet.langs()}")
        self.language = language

    def _get_replacement_words(self, word, random=False):
        """Returns a list containing all possible words with 1 character
        replaced by a homoglyph."""
        synonyms = set()
        for syn in wordnet.synsets(word, lang=self.language):
            for syn_word in syn.lemma_names(lang=self.language):
                if (
                    (syn_word != word)
                    and ("_" not in syn_word)
                    and (textattack.shared.utils.is_one_word(syn_word))
                ):
                    # WordNet can suggest phrases that are joined by '_' but we ignore phrases.
                    synonyms.add(syn_word)
        return list(synonyms)


In [30]:
class PWWSham2024INDO(AttackRecipe):
    
    @staticmethod
    def build(model_wrapper):
        transformation = WordSwapWordNetBahasa(language="ind")
        constraint = [RepeatModification(), StopwordModification()]
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR("weighted-saliency")
        return Attack(goal_function, constraint, transformation, search_method)

In [31]:
custom_data = list(df_test[['text', 'label']].itertuples(index=False, name=None))

In [32]:
from textattack.datasets import Dataset

dataset = Dataset(custom_data) #HuggingFaceDataset("tyqiangz/multilingual-sentiments", "indonesian", "test")
attack = PWWSham2024INDO.build(model_wrapper)

attackargs = AttackArgs(num_examples=500, log_to_csv="attack_results_classic.csv")
attacker = Attacker(attack, dataset, attack_args=attackargs)
attack_results = attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to C:\Users\Ilham
[nltk_data]     AM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'sklearn.naive_bayes.MultinomialNB'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_classic.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNetBahasa
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|          | 1/500 [00:00<07:23,  1.13it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[2 (81%)]] --> [[[SKIPPED]]]

kemarin gue datang ke tempat makan baru yang ada di dago atas gue kira makanan nya enak karena harga nya mahal ternyata boro boro tidak mau lagi deh ke tempat itu sudah mana tempat nya juga tidak nyaman banget terlalu sempit 




[Succeeded / Failed / Skipped / Total] 1 / 0 / 2 / 3:   1%|          | 3/500 [00:03<10:08,  1.22s/it]

...

--------------------------------------------- Result 482 ---------------------------------------------
[[2 (49%)]] --> [[[SKIPPED]]]

cerita si kancil itu identik dengan kecerdikan seorang kancil menyelesai masalah yang ada di hadapan nya 


--------------------------------------------- Result 483 ---------------------------------------------
[[0 (58%)]] --> [[[SKIPPED]]]

kalau mau beli obat obatan datang saja ke kimia farma karena di sana lengkap dan harga terjangkau 


--------------------------------------------- Result 484 ---------------------------------------------
[[0 (60%)]] --> [[[SKIPPED]]]

buah mangga yang kemarin gue lihat di pohon kayak nya sudah dimakan sama kelelawar 


--------------------------------------------- Result 485 ---------------------------------------------
[[0 (66%)]] --> [[[SKIPPED]]]

semua perempuan suka dikasih bunga atau puisi sama pacar nya 


--------------------------------------------- Result 486 ---------------------------------------------
[[

[Succeeded / Failed / Skipped / Total] 223 / 12 / 262 / 497:  99%|█████████▉| 497/500 [01:29<00:00,  5.55it/s]

--------------------------------------------- Result 489 ---------------------------------------------
[[2 (84%)]] --> [[[FAILED]]]

di bandung banyak tempat yang bisa didatangkan buat wisata sama keluarga yang tempat nya bagus bagus 


--------------------------------------------- Result 490 ---------------------------------------------
[[0 (68%)]] --> [[[SKIPPED]]]

indonesia punya banyak banget suku yang beragam budaya nya dan itu mencerminkan kalau negara kita sangat kaya 


--------------------------------------------- Result 491 ---------------------------------------------
[[0 (58%)]] --> [[[SKIPPED]]]

kalau jadi pemimpin itu harus bisa adil 


--------------------------------------------- Result 492 ---------------------------------------------
[[0 (47%)]] --> [[[SKIPPED]]]

jadi pemimpin di indonesia itu banyak tantangan nya 


--------------------------------------------- Result 493 ---------------------------------------------
[[2 (67%)]] --> [[[SKIPPED]]]

salah satu tempa

[Succeeded / Failed / Skipped / Total] 224 / 12 / 264 / 500: 100%|██████████| 500/500 [01:29<00:00,  5.57it/s]

--------------------------------------------- Result 498 ---------------------------------------------
[[1 (38%)]] --> [[2 (57%)]]

salah satu kegemaran anak remaja [[indonesia]] sekarang adalah mendengar kpop 

salah satu kegemaran anak remaja [[Nederlands-Indië]] sekarang adalah mendengar kpop 


--------------------------------------------- Result 499 ---------------------------------------------
[[0 (47%)]] --> [[[SKIPPED]]]

melihat warna hijau bisa bikin mata jadi lebih santai 


--------------------------------------------- Result 500 ---------------------------------------------
[[2 (54%)]] --> [[[SKIPPED]]]

bondan winarno yang suka bilang maknyus sekarang sudah tiada 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 224    |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 264    |
| Original accuracy:            | 47.2%  |
| Accurac

In [ ]:
import pandas as pd
from textattack.loggers import CSVLogger  # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult


pd.options.display.max_colwidth = (
    480  # increase colum width so we can actually read the examples
)

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv
C:\Users\Ilham AM\AppData\Local\Temp\ipykernel_12364\2054896546.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,original_text,perturbed_text
0,"kemarin gue datang ke tempat makan baru yang ada di dago atas . gue kira makanan nya enak karena harga nya mahal . ternyata , boro-boro . tidak mau lagi deh ke tempat itu . sudah mana tempat nya juga tidak nyaman banget , terlalu sempit .","kemarin gue datang ke tempat makan baru yang ada di dago atas . gue kira makanan nya afiat karena harga nya mahal . ternyata , boro-boro . tidak mau lagi deh ke tempat itu . sudah mana alun-alun nya juga tidak nyaman banget , terlalu sempit ."
1,"saya mesan tiket tapi tidak dikonformasi dan proses pengembalian uang yang tidak jelas dan belum ada sampai sekarang , pelayanan nya yang sangat buruk dan saran saya jangan pesan di sini , sangat mengecewakan .","saya mesan tiket tapi tidak dikonformasi dan proses pengembalian uang yang tidak jelas dan belum ada sampai sekarang , pelayanan nya yang sekali buruk dan saran saya jangan pesan di sini , sangat mengecewakan ."
2,"sangat kecewa dengan aplikasi ini , sudah ditransfer tetapi etiket tidak dikirim saya coba telepon call centre seharian tetapi sibuk terus call genre nya , mending pakai traveloka adalah untuk pesanan selanjutnya .","sekali kecewa dengan aplikasi ini , sudah ditransfer tetapi etiket tidak dikirim saya coba telepon call centre seharian tetapi sibuk terus call genre nya , mending pakai traveloka adalah untuk pesanan selanjutnya ."
3,melihat komen nya 90 % negatif jadi pikir-pikir buat mencari tiket kereta di sini . mending aplikasi yang lain saja yang sudah terbukti bagus bertahun-bertahun .,lihat komen nya 90 % negatif jadi pikir-pikir buat mencari tiket kereta di sini . mending aplikasi yang lain saja yang sudah terbukti Berguna bertahun-bertahun .
4,aplikasi tukang bohong . promo gratis makan tidak terjadi . komplain ke pelayanan pelanggan dengan email tidak ditanggapi . komplain by phone hanya dicatat saja tidak ada tindak lanjut . sangat tidak direkomendasikan untuk siapa pun .,aplikasi tukang bohong . promo gratis mau tidak terjadi . komplain ke pelayanan pelanggan dengan email tidak ditanggapi . komplain by phone hanya dicatat saja tidak ada tindak lanjut . sangat tidak direkomendasikan untuk siapa pun .
5,"tidak bisa menggunakan kode promo di aplikasi android , telpon ke pelayanan pelanggan bilang nya tidak ada solusi nya . oke uninstall dan tidak rekomendasi !","tidak bisa menggunakan kode promo di aplikasi android , telpon ke pelayanan pelanggan bilang nya tidak orang solusi nya . oke uninstall dan tidak cadangan !"
6,maaf ya saya uninstall . saya kecewa karena sudah melakukan pembayaran tetapi tidak dikonfirmasi . saya sudah komplain via email tetapi respon nya sangat lambat .,maaf ya saya uninstall . saya kecewa karena sudah melakukan pembayaran tetapi tidak dikonfirmasi . saya sudah komplain via email tetapi respon nya sekali lambat .
7,minta dikembalikan karena permasalahan bencana alam tapi tidak ada respon baik dari pelanggan nya . sangat mengecewakan .,minta dikembalikan karena permasalahan bencana alam tapi tidak ada respon baik dari pelanggan nya . sekali mengecewakan .
8,"terminal nya tidak ditulis tahu dari mana , saya kontak tidak ada balasan , mending aplikasi sebelah lebih oke , baru sekali coba sudah kayak gini kesan nya . tidak asik .","terminal nya tidak ditulis tahu dari mana , saya kontak tidak ada balasan , mending aplikasi sebelah lebih oke , baru sekali anju sudah kayak gini kesan nya . tidak asik ."
9,"kami meyewa 2 kamar sangat kecewa dengan fasilitas yang didapat , mulai dari ac yang tidak berfungsi dengan baik , alat cukur di 2 kamar yang tidak layak pakai sampai dapat apel yang busuk di evening coctail .","kami meyewa 2 kamar sangat kecewa dengan fasilitas yang didapat , mulai dari ac yang tidak berfungsi dengan afdal , alat cukur di 2 kamar yang tidak layak pakai sampai dapat apel yang busuk di evening coctail ."
